In [ ]:
import pandas as pd
from google.colab import files

# Upload the CSV file
uploaded = files.upload()

# Get the filename (assuming only one file is uploaded)
filename = list(uploaded.keys())[0]

# Load the CSV into a pandas DataFrame
drivers_df = pd.read_csv(filename)

# Display the first few rows
drivers_df.head(20)

Saving f1_fantasy_drivers.csv to f1_fantasy_drivers (1).csv


,driver,2024_points,cost
0,norris,374,29.0
1,verstappen,437,28.4
2,leclerc,356,25.9
3,hamilton,223,24.2
4,piastri,292,23.0
5,russell,245,21.0
6,antonelli,0,18.4
7,lawson,4,18.0
8,sainz,290,13.1
9,albon,12,12.0


In [ ]:
# Ensure there are no division errors (e.g., cost should not be zero)
drivers_df['value'] = drivers_df['points_2024'] / drivers_df['cost']

# Display the updated DataFrame
print(drivers_df)

        driver  2024_points  cost      value
0       norris          374  29.0  12.896552
1   verstappen          437  28.4  15.387324
2      leclerc          356  25.9  13.745174
3     hamilton          223  24.2   9.214876
4      piastri          292  23.0  12.695652
5      russell          245  21.0  11.666667
6    antonelli            0  18.4   0.000000
7       lawson            4  18.0   0.222222
8        sainz          290  13.1  22.137405
9        albon           12  12.0   1.000000
10       gasly           42  11.8   3.559322
11     tsunoda           30   9.6   3.125000
12      alonso           70   8.8   7.954545
13      stroll           24   8.1   2.962963
14        ocon           23   7.3   3.150685
15      doohan            0   7.2   0.000000
16     bearman            7   6.7   1.044776
17  hulkenberg           41   6.4   6.406250
18      hadjar            0   6.2   0.000000
19  bortoletto            0   6.0   0.000000


In [ ]:
# Upload the CSV file
uploaded = files.upload()

# Get the filename (assuming only one file is uploaded)
filename = list(uploaded.keys())[0]

# Load the CSV into a pandas DataFrame
teams_df = pd.read_csv(filename)

# Display the first few rows
teams_df.head()


Saving f1_fantasy_teams.csv to f1_fantasy_teams (2).csv


,team,2024_points,cost
0,mclaren,666,30.0
1,ferrari,652,27.1
2,red_bull,589,25.2
3,mercedes,468,22.7
4,aston_martin,94,8.5


In [ ]:
# Ensure there are no division errors (e.g., cost should not be zero)
teams_df['value'] = teams_df['points_2024'] / teams_df['cost']

# Display the updated DataFrame
print(teams_df)

           team  2024_points  cost      value
0       mclaren          666  30.0  22.200000
1       ferrari          652  27.1  24.059041
2      red_bull          589  25.2  23.373016
3      mercedes          468  22.7  20.616740
4  aston_martin           94   8.5  11.058824
5        alpine           65   9.5   6.842105
6          haas           58   7.0   8.285714
7  racing_bulls           46   8.0   5.750000
8      williams           17  13.1   1.297710
9   kick_sauber            4   6.2   0.645161


In [ ]:
import itertools

def find_best_combinations(drivers_df, teams_df, max_cost=100, top_n=5):
    # Rename '2024_points' column to 'points' for easier access
    drivers_df = drivers_df.rename(columns={'points_2024': 'points'})
    teams_df = teams_df.rename(columns={'points_2024': 'points'})

    best_combinations = []

    # Generate all possible combinations of 5 drivers
    driver_combinations = list(itertools.combinations(drivers_df.itertuples(index=False), 5))
    # Generate all possible combinations of 2 teams
    team_combinations = list(itertools.combinations(teams_df.itertuples(index=False), 2))

    all_combinations = []

    for driver_combo in driver_combinations:
        for team_combo in team_combinations:
            # Compute total cost and points
            total_cost = sum(driver.cost for driver in driver_combo) + sum(team.cost for team in team_combo)
            total_points = sum(driver.points for driver in driver_combo) + sum(team.points for team in team_combo)

            # Check if within budget
            if total_cost <= max_cost:
                # Calculate individual contributions as percentage of total points
                driver_percentages = [(driver, driver.points / total_points * 100) for driver in driver_combo]
                team_percentages = [(team, team.points / total_points * 100) for team in team_combo]

                all_combinations.append((driver_percentages, team_percentages, total_points, total_cost))

    # Sort combinations by points in descending order
    all_combinations.sort(key=lambda x: x[2], reverse=True)

    # Return the top N combinations
    return all_combinations[:top_n]

def pretty_print_combinations(combinations):
    for i, (drivers, teams, points, cost) in enumerate(combinations, start=1):
        print(f"Combination {i}:")
        print("  Drivers:")
        for driver, percentage in drivers:
            print(f"    - {driver.driver} (Points: {driver.points}, Cost: {driver.cost}, Contribution: {percentage:.2f}%)")
        print("  Teams:")
        for team, percentage in teams:
            print(f"    - {team.team} (Points: {team.points}, Cost: {team.cost}, Contribution: {percentage:.2f}%)")
        print(f"  Total Points: {points}")
        print(f"  Total Cost: {cost}")
        print("-" * 40)

best_combos = find_best_combinations(drivers_df, teams_df, top_n=200)
pretty_print_combinations(best_combos)

Combination 1:
  Drivers:
    - sainz (Points: 290, Cost: 13.1, Contribution: 16.58%)
    - alonso (Points: 70, Cost: 8.8, Contribution: 4.00%)
    - ocon (Points: 23, Cost: 7.3, Contribution: 1.32%)
    - bearman (Points: 7, Cost: 6.7, Contribution: 0.40%)
    - hulkenberg (Points: 41, Cost: 6.4, Contribution: 2.34%)
  Teams:
    - mclaren (Points: 666, Cost: 30.0, Contribution: 38.08%)
    - ferrari (Points: 652, Cost: 27.1, Contribution: 37.28%)
  Total Points: 1749
  Total Cost: 99.4
----------------------------------------
Combination 2:
  Drivers:
    - sainz (Points: 290, Cost: 13.1, Contribution: 16.64%)
    - alonso (Points: 70, Cost: 8.8, Contribution: 4.02%)
    - stroll (Points: 24, Cost: 8.1, Contribution: 1.38%)
    - hulkenberg (Points: 41, Cost: 6.4, Contribution: 2.35%)
    - hadjar (Points: 0, Cost: 6.2, Contribution: 0.00%)
  Teams:
    - mclaren (Points: 666, Cost: 30.0, Contribution: 38.21%)
    - ferrari (Points: 652, Cost: 27.1, Contribution: 37.41%)
  Total Poin

In [ ]:
import itertools

def generate_combinations_with_mandatory(drivers_df, teams_df, required_drivers, required_teams, max_cost=100, top_n=10):
    """
    Generates and returns the top N valid combinations that include ALL required drivers and teams, while staying within budget.

    :param drivers_df: DataFrame containing driver data
    :param teams_df: DataFrame containing team data
    :param required_drivers: List or set of required driver names
    :param required_teams: List or set of required team names
    :param max_cost: Maximum allowed cost
    :param top_n: Number of top results to return
    :return: List of valid combinations sorted by points
    """
    # Rename '2024_points' column to 'points' for easier access
    drivers_df = drivers_df.rename(columns={'points_2024': 'points'})
    teams_df = teams_df.rename(columns={'points_2024': 'points'})

    # Convert required drivers and teams to sets for faster lookup
    required_drivers = set(required_drivers)
    required_teams = set(required_teams)

    # Separate required drivers and teams from the available pool
    selected_drivers = [driver for driver in drivers_df.itertuples(index=False) if driver.driver in required_drivers]
    selected_teams = [team for team in teams_df.itertuples(index=False) if team.team in required_teams]

    # Ensure all required drivers and teams exist
    if len(selected_drivers) != len(required_drivers) or len(selected_teams) != len(required_teams):
        print("Error: Some required drivers or teams are missing from the data.")
        return []

    # Compute base cost and points for required drivers and teams
    base_cost = sum(driver.cost for driver in selected_drivers) + sum(team.cost for team in selected_teams)
    base_points = sum(driver.points for driver in selected_drivers) + sum(team.points for team in selected_teams)

    # Remaining budget and slots
    remaining_budget = max_cost - base_cost
    remaining_driver_slots = 5 - len(selected_drivers)  # 5 total drivers
    remaining_team_slots = 2 - len(selected_teams)  # 2 total teams

    # Ensure the selection fits within budget constraints
    if remaining_budget < 0:
        print("Error: The required drivers and teams exceed the budget.")
        return []

    best_combinations = []

    # Generate all possible additional driver combinations
    additional_driver_options = [driver for driver in drivers_df.itertuples(index=False) if driver.driver not in required_drivers]
    driver_combinations = list(itertools.combinations(additional_driver_options, remaining_driver_slots))

    # Generate all possible additional team combinations
    additional_team_options = [team for team in teams_df.itertuples(index=False) if team.team not in required_teams]
    team_combinations = list(itertools.combinations(additional_team_options, remaining_team_slots))

    # Generate valid combinations
    for driver_combo in driver_combinations:
        for team_combo in team_combinations:
            total_cost = base_cost + sum(driver.cost for driver in driver_combo) + sum(team.cost for team in team_combo)
            total_points = base_points + sum(driver.points for driver in driver_combo) + sum(team.points for team in team_combo)

            if total_cost <= max_cost:
                # Calculate individual contributions as percentage of total points
                driver_percentages = [(driver, driver.points / total_points * 100) for driver in selected_drivers + list(driver_combo)]
                team_percentages = [(team, team.points / total_points * 100) for team in selected_teams + list(team_combo)]

                best_combinations.append((driver_percentages, team_percentages, total_points, total_cost))

    # Sort by points in descending order
    best_combinations.sort(key=lambda x: x[2], reverse=True)

    return best_combinations[:top_n]

required_drivers = ["norris", "antonelli"]
required_teams = ["ferrari", "williams"]

best_combos = generate_combinations_with_mandatory(drivers_df, teams_df, required_drivers, required_teams, top_n=10)
if len(best_combos) == 0:
    print("No valid combinations")
else:
    pretty_print_combinations(best_combos)

No valid combinations
